# Квадратная решётка. В каждой ячейке с вероятностью $p$ находится частица. Частицы образуют кластеры (учитываются связи только по горизонтали или вертикали). Если есть путь, соединяющий верхний ряд с нижним, то такой кластер называем бесконечным.

### Задача: определить вероятность появления бесконечного кластера, то есть значение $p$, при котором бесконечный кластер появляется. $P(p)=\frac{1}{2}, p-?$ Для каждого $p$ нужно провести много испытаний и вычислить вероятность появления бесконечного кластера. 

In [1]:
#import appropriate libraries for calculations
import numpy as np

In [2]:
#probabiliy - probability of node in lattice
def generate(probability):
    return np.random.choice([0, 1], p=[1-probability, probability])

In [3]:
#Algorithm calculating all the clusters present in lattice
#It returns lattice and list of clusters (set of pairs)
def lattice_with_its_clusters(probability, N = 10):

    clusters = []
    lattice = np.zeros((N, N))

    def insert_node(x_cond, y_cond, x, y):
        for k in range(len(clusters)):
            if((x_cond, y_cond) in clusters[k]):
                clusters[k].add((x, y))
                break

    for i in range(N):
        for j in range(N):
            lattice[i, j] = generate(probability)

            if(lattice[i, j] == 1): #begin to check neighbours and app
                if(j == 0 and i == 0): #First case when i = j = 0
                    clusters.append(set())
                    clusters[0].add((i, j))
                elif(j > 0 and i == 0): #Second case when i = 0, j > 0
                    if (lattice[i, j-1] == 0):
                        clusters.append(set())
                        clusters[-1].add((i, j))
                    else:
                        insert_node(i, j-1, i, j)
                elif(j == 0 and i > 0): #Third case when i > 0, j = 0
                    if(lattice[i-1, j] == 0):
                        clusters.append(set())
                        clusters[-1].add((i, j))
                    else:
                        insert_node(i-1, j, i, j)
                else: #Last case when i > 0, j > 0
                    if(lattice[i, j-1] == 0 and lattice[i-1, j] == 0): #First case when 0, 1, 0 (left, (i,j), upper)
                        clusters.append(set())
                        clusters[-1].add((i, j))
                    elif(lattice[i, j-1] == 1 and lattice[i-1, j] == 0): #Second case when 1, 1, 0
                        insert_node(i, j-1, i, j)
                    elif(lattice[i, j-1] == 0 and lattice[i-1, j] == 1): #Third case when 0, 1, 1
                        insert_node(i-1, j, i, j)
                    else: #Fourth case when 1, 1, 1 splitted in 2 subcases
                        for k in range(len(clusters)):
                            if((i, j-1) in clusters[k]):
                                clusters[k].add((i, j)) #When neighbours are not in one cluster then:
                                if((i-1, j) not in clusters[k]): 
                                    clusters[k].add((i, j))
                                    for kk in range(len(clusters)): #merge sets via (i,j) node
                                        if((i-1, j) in clusters[kk]):
                                            for item in clusters[k]:
                                                clusters[kk].add(item)
                                            del clusters[k]
                                            break
                                    break
    return lattice, clusters

In [4]:
#Algorithm checking if there is an infinite cluster in given lattice
def has_infinite_cluster(clusters, N=10):
    for cluster in clusters:
        if(len(cluster) >= 10): #size less than number of nodes in length cannot create infinite cluster
            first = 0
            last = 0
            for node in cluster:
                if(node[0] == 0):
                    first = 1
                if(node[0] == N-1):
                    last = 1
            if(first == 1 and last == 1):
                return True
    return False

In [5]:
#Example of algorithm work
lat, clut = lattice_with_its_clusters(probability=0.48, N=10)
has_infinite_cluster(clut, N=10)

False

In [6]:
#Prited lattice and all clusters in it
print(lat)
clut

[[0. 1. 0. 0. 1. 0. 0. 0. 0. 0.]
 [1. 0. 1. 1. 1. 0. 1. 0. 0. 0.]
 [1. 0. 1. 0. 0. 0. 1. 1. 0. 0.]
 [1. 0. 1. 0. 0. 1. 0. 1. 0. 0.]
 [1. 1. 0. 1. 0. 0. 1. 1. 1. 0.]
 [1. 1. 0. 0. 0. 1. 0. 0. 1. 0.]
 [0. 0. 0. 1. 1. 1. 0. 1. 0. 1.]
 [0. 1. 1. 0. 1. 0. 0. 0. 1. 1.]
 [1. 0. 0. 0. 1. 1. 0. 0. 1. 1.]
 [1. 1. 0. 0. 0. 1. 1. 0. 1. 0.]]


[{(0, 1)},
 {(0, 4), (1, 2), (1, 3), (1, 4), (2, 2), (3, 2)},
 {(1, 0), (2, 0), (3, 0), (4, 0), (4, 1), (5, 0), (5, 1)},
 {(1, 6), (2, 6), (2, 7), (3, 7), (4, 6), (4, 7), (4, 8), (5, 8)},
 {(3, 5)},
 {(4, 3)},
 {(5, 5), (6, 3), (6, 4), (6, 5), (7, 4), (8, 4), (8, 5), (9, 5), (9, 6)},
 {(6, 7)},
 {(6, 9), (7, 8), (7, 9), (8, 8), (8, 9), (9, 8)},
 {(7, 1), (7, 2)},
 {(8, 0), (9, 0), (9, 1)}]

In [7]:
#Make the experiment with given parameters of p (probabality) and N (size of lattice)
#number_of_iterations times to define the probability of
#appearance of infinite cluster
def P_probability_of_infinite_cluster(number_of_iterations, probability, N=10):
    infinite_clusters_true = 0;
    for i in range(number_of_iterations):
        lat, clut = lattice_with_its_clusters(probability, N)
        if(has_infinite_cluster(clut, N)):
            infinite_clusters_true += 1
    return infinite_clusters_true / number_of_iterations

# Посчитаем (1000 раз для каждой вероятности и размера решетки) вероятность для размеров решёток $N\in\{5, 10, 15, 30, 100\}$ в промежутке вероятности $p\in[0.3, 0.8]$, а также уточнённую версию для $N=30$ в промежутке вероятности $p\in[0.58,0.6]$

## Все результаты записываются в файлы (чтобы было потом удобнее это считывать, а не каждый раз запускать программу: долго считается) 

In [8]:
#This piece of code calclucated all the appropriated data

#for N_lat in [5, 10, 15, 30, 100]:
#    lattice_prob_array = np.linspace(0.3, 0.8, 501)
#    P = []
#    p = []
#    for k in lattice_prob_array:
#        print(k)
#        p.append(k)
#        P.append(P_probability_of_infinite_cluster(1000, k, N=N_lat))
#    np.save('inf_clust_prob_30_' + str(N_lat, P)
#np.save('lattice_prob.npy', np.linspace(0.3, 0.8, 501))


#P = []
#p = []
#lattice_prob_array = np.linspace(0.58, 0.6, 201)
#for k in lattice_prob_array:
#    print(k)
#    p.append(k)
#    P.append(P_probability_of_infinite_cluster(10000, k, N=30))
#np.save('inf_clust_prob_30_thorough_10000.npy', P)
#np.save('lattice_prob_10000.npy', np.linspace(0.58, 0.6, 201))